In [2]:
import numpy as np
import pandas as pd
import xlrd
from functools import reduce
import altair as alt

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('Data/gun_violence.csv')

In [4]:
df.head()

,Unnamed: 0,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Category,Age
0,0,"November 26, 2016",Maryland,Sabillasville,6600 block of Eylers Valley Flint Road,1,0,NaN,Accidental,Adult
1,1,"November 26, 2016",Ohio,Grand Rapids,SR Route 65,1,0,NaN,Accidental,Adult
2,2,"November 24, 2016",New York,Sandy Creek,Kehoe Road,1,0,NaN,Accidental,Adult
3,3,"November 24, 2016",Florida,Black Point,NaN,1,0,NaN,Accidental,Adult
4,4,"November 23, 2016",Michigan,Powell Township,North 1311 County Road KCI,1,0,NaN,Accidental,Adult


We first create a dataframe that removes all duplicate entries so that we can get the total count of incidents.

In [5]:
df[df.duplicated(['Incident Date','Address'])].sort_values(['Incident Date', 'Address'], ascending=True)

,Unnamed: 0,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Category,Age
5378,421,"April 10, 2016",Wisconsin,Milwaukee,76th and Sheridan,1,3,NaN,Intentional,Teen
770,64,"April 10, 2016",North Carolina,Granite Falls,Baton Church Road,1,0,NaN,Accidental,Teen
5377,420,"April 10, 2016",North Carolina,Granite Falls,Baton Church Road,1,0,NaN,Intentional,Teen
3446,483,"April 12, 2014",South Carolina,Gaston,Wild Bird Lane,1,1,NaN,Intentional,Child
3279,316,"April 12, 2015",Ohio,Cleveland,3700 block of E. 63,1,0,NaN,Intentional,Child
3445,482,"April 13, 2014",North Carolina,Charlotte,700 Glasgow Road,1,0,NaN,Intentional,Child
2763,300,"April 13, 2016",Mississippi,Jackson,1850 Chadwick Dr,0,1,NaN,Intentional,Child
769,63,"April 14, 2016",California,Victorville,13000 block of Bennington Drive,1,0,NaN,Accidental,Teen
5373,416,"April 14, 2016",New Jersey,Orange,Taylor Street,1,3,NaN,Intentional,Teen
557,57,"April 16, 2016",Pennsylvania,Philadelphia,200 block of E. Mayfield Street,1,0,NaN,Accidental,Child


In [6]:
df.drop(['Unnamed: 0'], axis=1).set_index(['Incident Date','Address']).sort_values('Incident Date', ascending=True).head(100)

State  \
Incident Date  Address                                                 
April 1, 2015  1600 block of 1st Street N.W                  Alabama   
               18600 block of East 49th Street              Oklahoma   
April 1, 2016  3700 block of Macinac Island lane      North Carolina   
               Wooddale Avenue and Boxdale Street          Tennessee   
April 10, 2014 Roswell Drive                                    Ohio   
April 10, 2015 2600 block of Durby Circle                  Tennessee   
               11290 Fondren                                   Texas   
               2800 block of  Venetian Drive                 Florida   
April 10, 2016 76th and Sheridan                           Wisconsin   
               84th Street and Towne Avenue               California   
               Walder Road                               Mississippi   
               Hillsdale Terrace                            Arkansas   
               5700 block of Elk Street                   California   
               Baton Church Road                      North Carolina   
               76th and Sheridan                           Wisconsin   
               Baton Church Road                      North Carolina   
               29000 block of Weld County Road 20           Colorado   
               Baton Church Road                      North Carolina   
April 11, 2014 North 15th and H Streets                      Indiana   
April 11, 2016 NaN                                         Wisconsin   
               3400 block of West Grenshaw Street           Illinois   
               Green Bay Avenue and Capitol Drive          Wisconsin   
               1800 block of South 19th Street          Pennsylvania   
April 12, 2014 Wild Bird Lane                         South Carolina   
               Wild Bird Lane                         South Carolina   
               2100 block of Pauger Street                 Louisiana   
April 12, 2015 3700 block of E. 63                              Ohio   
               3700 block of E. 63                              Ohio   
April 12, 2016 8500 block of Riverview                      Missouri   
               200 block of Colt Drive                         Texas   
...                                                              ...   
April 17, 2015 100 block of Carroll Street                 Louisiana   
April 17, 2016 2100 block of Lima Center Road              Wisconsin   
               700 College Street                          Tennessee   
               McLemore Avenue and Third Street            Tennessee   
               200 block of Eastview Road             South Carolina   
               1900 block of Greenbrook Lane                Michigan   
               NaN                                    North Carolina   
               400 block of North Church Street           California   
               5021 W St Hwy 107                               Texas   
               6500 block of West Mountain View Road         Arizona   
               600 block of Westmoreland Street         Pennsylvania   
               400 block of North Church Street           California   
April 18, 2014 3616 West 2200 South                             Utah   
               North Saunders Avenue                        Nebraska   
April 18, 2015 NaN                                          Virginia   
               Rosa Parks Blvd and Godwin Ave             New Jersey   
               3600 block of Del Mar Court                     Texas   
               1800 block of Gibbs Court                     Alabama   
               3600 block of Del Mar Court                     Texas   
               Bost Street                            North Carolina   
               321 Pine St                              Pennsylvania   
               Martin Luther King Drive               North Carolina   
April 18, 2016 1200 block of E. 17th Street               California   
               Interstate 4    

Before we begin, let's have a look at the general distribution of the data.

# Is there a correlation between Police involvement and crimes also involving children and teens?

# Is there seasonality to gun violence?

We look at the pattern for each incident category

In [8]:
import datetime as dt

Convert the `Incident Date` column to datetime type

In [9]:
df['Incident Date'] = pd.to_datetime(df['Incident Date'], format="%B %d, %Y")

In [10]:
df.head()

,Unnamed: 0,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Category,Age
0,0,2016-11-26,Maryland,Sabillasville,6600 block of Eylers Valley Flint Road,1,0,NaN,Accidental,Adult
1,1,2016-11-26,Ohio,Grand Rapids,SR Route 65,1,0,NaN,Accidental,Adult
2,2,2016-11-24,New York,Sandy Creek,Kehoe Road,1,0,NaN,Accidental,Adult
3,3,2016-11-24,Florida,Black Point,NaN,1,0,NaN,Accidental,Adult
4,4,2016-11-23,Michigan,Powell Township,North 1311 County Road KCI,1,0,NaN,Accidental,Adult


In [11]:
df['Quarter'] = df['Incident Date'].dt.quarter
df['Year'] = df['Incident Date'].dt.year
df.head(5)

,Unnamed: 0,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Category,Age,Quarter,Year
0,0,2016-11-26,Maryland,Sabillasville,6600 block of Eylers Valley Flint Road,1,0,NaN,Accidental,Adult,4,2016
1,1,2016-11-26,Ohio,Grand Rapids,SR Route 65,1,0,NaN,Accidental,Adult,4,2016
2,2,2016-11-24,New York,Sandy Creek,Kehoe Road,1,0,NaN,Accidental,Adult,4,2016
3,3,2016-11-24,Florida,Black Point,NaN,1,0,NaN,Accidental,Adult,4,2016
4,4,2016-11-23,Michigan,Powell Township,North 1311 County Road KCI,1,0,NaN,Accidental,Adult,4,2016


In [12]:
df.Category.unique()

array(['Accidental', 'Intentional', 'Mass Shootings',
       'Police Involvement'], dtype=object)

In [13]:
df[df["Category"] == "Mass Shootings"].groupby(["Year","Quarter"]).count()

Unnamed: 0  Incident Date  State  City Or County  Address  \
Year Quarter                                                              
2014 1                50             50     50              50       47   
     2                69             69     69              69       68   
     3                92             92     92              92       88   
     4                64             64     64              64       64   
2015 1                60             60     60              60       60   
     2                91             91     91              91       89   
     3               115            115    115             115      114   
     4                67             67     67              67       67   
2016 1                60             60     60              60       60   
     2               108            108    108             108      108   
     3               123            123    123             123      123   
     4                60             60     60              60       59   

              # Killed  # Injured  Operations  Category  Age  
Year Quarter                                                  
2014 1              50         50           0        50    0  
     2              69         69           0        69    0  
     3              92         92           0        92    0  
     4              64         64           0        64    0  
2015 1              60         60           0        60    0  
     2              91         91           0        91    0  
     3             115        115           0       115    0  
     4              67         67           0        67    0  
2016 1              60         60           0        60    0  
     2             108        108           0       108    0  
     3             123        123           0       123    0  
     4              60         60           0        60    0

In [14]:
%%HTML
<div class='tableauPlaceholder' id='viz1558744621263' style='position: relative'>
	<noscript><a href='#'>
	<img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gu&#47;GunViolenceAnalysis_15587372850870&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript>
	<object class='tableauViz'  style='display:none;'>
		<param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> 
		<param name='embed_code_version' value='3' /> 
		<param name='site_root' value='' /><param name='name' value='GunViolenceAnalysis_15587372850870&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' />
		<param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Gu&#47;GunViolenceAnalysis_15587372850870&#47;Dashboard1&#47;1.png' /> 
		<param name='animate_transition' value='yes' />
		<param name='display_static_image' value='yes' />
		<param name='display_spinner' value='yes' />
		<param name='display_overlay' value='yes' />
		<param name='display_count' value='yes' />
	</object>
</div>
<script type='text/javascript'>
	var divElement = document.getElementById('viz1558744621263');
	var vizElement = divElement.getElementsByTagName('object')[0];
	if ( divElement.offsetWidth > 800 ) { 
		vizElement.style.width='1000px';vizElement.style.height='827px';
	} else if ( divElement.offsetWidth > 500 ) { 
		vizElement.style.width='1000px';vizElement.style.height='827px';
	} else { 
		vizElement.style.width='100%';vizElement.style.height='727px';
	}
	var scriptElement = document.createElement('script');
	scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';
	vizElement.parentNode.insertBefore(scriptElement, vizElement);
</script>

Looking at the chart, we can see that:
<ol>
    <li>There is an increasing trend to gun violence incidents in the United States between 20014 - 2018.</li>
    <li>There is a sudden spike in 2016 Q3 in Accidental and Intentional incidents from previous quarters.</li>
    <li>There is a higher occurence of Mass Shootings in Q3 compared to other quarters in all years.</li>
    <li>Accidental gun violence incidents are more common in the second half (July - December) of the year.</li>
    <li>Beginning March 2014, there is at least 1 reported incident of gun violence in the United States each month.</li>
</ol>
    

# Frequency of police involvement in acts of gun violence?